## AWQ Mistral


In [ ]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch
import os
from prompts import classifier_prompt

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

In [ ]:
weights_dir = "./weights"
model_name_or_path = "TheBloke/Mistral-7B-v0.1-AWQ"

# Load model
model = AutoAWQForCausalLM.from_quantized(
    model_name_or_path,
    fuse_layers=True,
    trust_remote_code=False,
    safetensors=True,
    device=device,
    cache_dir=weights_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=False, device=device, cache_dir=weights_dir
)

### Test model


In [ ]:
user_query = "Move robot tcp left for 1000mm"
prompt_template = f"""{classifier_prompt.format(user_query=user_query)}"""

print("\n\n*** Generate:")

tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens, do_sample=True, temperature=0.1, top_p=0.95, top_k=40, max_new_tokens=30
)

print("Output: ", tokenizer.decode(generation_output[0]))

### Test pipeline


In [ ]:
# Inference should be possible with transformers pipeline as well in future
# But currently this is not yet supported by AutoAWQ (correct as of September 25th 2023)
from transformers import pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1,
    device=device,
)

print(pipe(prompt_template)[0]["generated_text"])

## GPTQ Mistral


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from prompts import *

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

In [ ]:
weights_dir = "./weights"
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto",
    trust_remote_code=False,
    cache_dir=weights_dir,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path, trust_remote_code=False, device=device, cache_dir=weights_dir
)

### Test model


In [ ]:
user_query = ("Rotate robot base for 45 and move TCP along x axis for 50 milimeters.",)

prompt_template = f"""{classifier_prompt_1.format(user_query=user_query)}"""

print("\n\n*** Generate:")

tokens = tokenizer(prompt_template, return_tensors="pt").input_ids.cuda()

# Generate output
generation_output = model.generate(
    tokens, do_sample=True, temperature=0.1, top_p=0.95, top_k=40, max_new_tokens=512
)

print("Output: ", tokenizer.decode(generation_output[0]))

## JSON former


In [ ]:
from jsonformer import Jsonformer
import json

In [ ]:
user_query_tests = [
    "Move robot tcp left for 1000mm",
    "Move robot sixth joint for 30 degrees left",
    "Give me robot joint info",
    "Tell me info about robot",
    "Rotate robot base for 45 and move TCP along x axis for 50 milimeters.",
    "Rotate joint 2 for 30 and joint 7 for 45 degrees",
]

In [ ]:
user_query_tests[4]

In [ ]:
json_schema = {
    "type": "object",
    "properties": {
        "function_name": {"type": "array", "items": {"type": "string"}},
    },
}

prompt_template = f"""{classifier_prompt_1.format(user_query=user_query_tests[5])}"""
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt_template)
generated_data = jsonformer()

print(generated_data)

In [ ]:
json_schema = {
    "type": "object",
    "properties": {
        "functions": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "function_name": {"type": "string"},
                    "inputs": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "input_name": {
                                    "type": "string",
                                },
                                "input_value": {"type": "string"},
                            },
                        },
                    },
                },
            },
        },
    },
}

prompt_template = f"""{classifier_prompt_1.format(user_query=user_query_tests[4])}"""
jsonformer = Jsonformer(model, tokenizer, json_schema, prompt_template)
generated_data = jsonformer()

print(json.dumps(generated_data, indent=4))